# Inicjalizacja kontekstu użytkownika

**Cel**: Konfiguracja środowiska i izolacja zasobów.

Skrypt realizuje strategię **Catalog Isolation** (rekomendowana):
1.  Tworzy dedykowany katalog dla użytkownika: `ecommerce_platform_<user_slug>` (np. `ecommerce_platform_kzb`).
2.  Wewnątrz katalogu tworzy standardowe schematy warstw: `bronze`, `silver`, `gold`.
3.  Ustawia zmienne globalne (`CATALOG`, `BRONZE_SCHEMA` itp.) używane w notebookach.

*Jeśli użytkownik nie ma uprawnień do tworzenia katalogów, skrypt automatycznie przełączy się na tryb izolacji schematami w katalogu `ecommerce_platform`.*

**Uruchom tę komórkę raz na początku sesji.**

## Architektura danych w szkoleniu

Szkolenie używa **volume  do przechowywania danych** :

- **Źródło**: Unity Catalog Volumes (`/Volumes/ecommerce_platform_<user>/default/kion_datasets`)
- **Zmienna**: `VOLUMES_BASE_PATH` (opcjonalne dla Dzień 3)
- **Cel**: Demonstracja zaawansowanych feature'ów UC (DLT, Lakeflow, Governance)
- **Przykład**: `spark.read.csv("/Volumes/ecommerce_platform_kzb/default/kion_datasets/customers.csv")`

---

In [0]:
# === Konfiguracja katalogów i schematów ===
# Pobierz aktualnego użytkownika (dla środowiska produkcyjnego)
# raw_user = spark.sql("SELECT current_user()").first()[0]
raw_user = "trainer"  # Dla środowiska szkoleniowego

import re
user_slug = re.sub(
    r'[^a-z0-9]', 
    '_', 
    raw_user.lower()
)


# Opcja 1: Izolacja Katalogami (Czysty kod: FROM bronze.orders)
# Wymaga uprawnień CREATE CATALOG
CATALOG = f"ecommerce_platform_{user_slug}"
BRONZE_SCHEMA = "bronze"
SILVER_SCHEMA = "silver"
GOLD_SCHEMA   = "gold"
ISOLATION_MODE = "Catalog"


 #Dla Databricks środowiska - Unity Catalog Volume:
# Zakładamy, że dane źródłowe są we wspólnym katalogu 'training_sources' lub podobnym
# Tutaj dla uproszczenia używamy katalogu użytkownika, ale w realnym scenariuszu byłby to read-only shared volume
DATASET_BASE_PATH = f"/Volumes/ecommerce_platform_trainer/default/kion_datasets"


print(f"Dataset base path: {DATASET_BASE_PATH}")

spark.sql(f'USE CATALOG {CATALOG}')
    
for s in [BRONZE_SCHEMA, SILVER_SCHEMA, GOLD_SCHEMA]:
    spark.sql(f'CREATE SCHEMA IF NOT EXISTS {CATALOG}.{s}')
    
# Opcjonalnie: Utwórz wolumen na dane jeśli nie istnieje (dla celów szkoleniowych)
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.default")
# spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.default.kion_datasets") 
    

display(
    spark.sql(f"DESCRIBE CATALOG EXTENDED {CATALOG}")
)


print(f"✓ User slug: {user_slug}")
print(f"✓ Tryb izolacji: {ISOLATION_MODE}")
print(f"✓ Katalog roboczy: {CATALOG}")
print(f"✓ Schematy: {BRONZE_SCHEMA}, {SILVER_SCHEMA}, {GOLD_SCHEMA}")
